In [ ]:
!pip install unidecode
from unidecode import unidecode
import pandas as pd
import numpy as np
import datetime as dt
from datetime import datetime
import re
from openpyxl import Workbook, styles
from openpyxl.styles import PatternFill, Font
from openpyxl.styles.differential import DifferentialStyle
from openpyxl.formatting.rule import Rule

TowerDB Comparisons

In [ ]:
def find_diffs_between_files_tw(path_OLD, path_NEW, index_col, bill_cols, status_col, path_save, old_name, new_name, type_file='mix',\
                                kind='tw', sheetname='',dates=[], skipr=0, skipc=0):
    
    def lower_str(columns):
        newlist = list(map(lambda x: x.lower(), columns))
        return newlist

    def fit_df(df, bill_cols):
        fit_cols = lower_str(list(df.columns))
        df.columns = fit_cols
        #df = df[bill_cols]
        df = df.dropna(subset=[index_col], axis=0)
        #df = df.drop_duplicates(subset=[index_col], keep='last')
        df[index_col] = df[index_col].astype(str)
        df['sites'] = df[index_col]
        # Aqui ajusta os nan e nat dos DFs, quando não forem arquivos não lidos
        df = df.set_index('sites').fillna('')
        return df
        
    def change_format(index, worksheet, format):
        for cell in worksheet[f"{str(index)}:{str(index)}"]:
            cell.font = format
              
    bill_cols = lower_str(bill_cols)
    index_col = index_col.lower()
    type_file = type_file.lower()
    status_col = status_col.lower()
    if type_file == 'excel':
        df_OLD = pd.read_excel(path_OLD,sheet_name = sheetname, skiprows = skipr).fillna('')
        df_OLD = df_OLD.iloc[:,skipc:]
        df_OLD = fit_df(df_OLD,bill_cols)

        df_NEW = pd.read_excel(path_NEW,sheet_name = sheetname, skiprows = skipr).fillna('')
        df_NEW = df_NEW.iloc[:,skipc:]
        df_NEW = fit_df(df_NEW,bill_cols)

    elif type_file == 'csv':
        df_OLD = pd.read_csv(path_OLD, encoding='latin').fillna('')
        df_OLD = fit_df(df_OLD,bill_cols)

        df_NEW = pd.read_csv(path_NEW, encoding='latin').fillna('')
        df_NEW = fit_df(df_NEW,bill_cols)

    else:
        df_OLD = pd.read_csv(path_OLD, encoding='latin')
        df_OLD = fit_df(df_OLD,bill_cols)

        df_NEW = pd.read_excel(path_NEW,sheet_name = sheetname, skiprows = skipr)
        df_NEW = df_NEW.iloc[:,skipc:]
        #date_parser(df_NEW, dates, 'normal', "%d/%m/%Y")
        df_NEW = fit_df(df_NEW,bill_cols)
        
        lista=[]
        for column in dates:
            df_NEW[column] = df_NEW[column].replace([0, '#N/D', 'N/A'],'')
            for dt in df_NEW[column]:
                if dt != '':
                    new_dt = f"{dt:%d/%m/%Y}"
                    lista.append(new_dt)
                else:
                    lista.append(dt)

            df_NEW[column] = lista

    # Perform Diff
    new_copy = df_NEW.copy()
    droppedRows = []
    newRows = []

    cols_OLD = df_OLD.columns
    cols_NEW = df_NEW.columns
    sharedCols = list(set(cols_OLD).intersection(cols_NEW))

    for row in new_copy.index:
        if (row in df_OLD.index) and (row in df_NEW.index):
            for col in sharedCols:
                value_OLD = str(df_OLD.loc[row,col])
                value_NEW = str(df_NEW.loc[row,col])
               #Error de a.empty() são sites duplcados e nã poodem estar no index
                if value_OLD == value_NEW:
                    new_copy.loc[row,col] = np.nan
                else:
                    new_copy.loc[row,col] = f'{value_OLD} > {value_NEW}'
        else:
            newRows.append(row)

    new_copy = new_copy.dropna(axis=0, how='all')
    new_copy = new_copy.dropna(axis=1, how='all')

    for row in df_OLD.index:
        if row not in df_NEW.index:
            droppedRows.append(row)
            new_copy = new_copy.append(df_OLD.loc[row,:])
    
    new_copy = new_copy.sort_index(key=lambda x: x.str.lower()).fillna('')
    
    new_copy = new_copy.reset_index()
    if kind=='tw':
        sites = [i for i in new_copy['sites']] 
        old = df_OLD[[status_col]].reset_index()
        old = old.loc[old['sites'].isin(sites)]
        new = df_NEW[[status_col]].reset_index()
        new = new.loc[new['sites'].isin(sites)]
        df_cross = pd.merge(new, old, on=['sites'], how='inner', suffixes=('_current', '_before'))
        new_copy = pd.merge(new_copy, df_cross, on=['sites'], how='left')
        status_1 = f'{status_col}_current'
        status_2 = f'{status_col}_before'
        new_copy = new_copy.set_index('sites')
        new_copy = new_copy[[status_2, status_1]+ new_copy.columns[:-2].tolist()]
        new_copy = new_copy.reset_index()


    print(f'\nNew Rows:     {newRows}')
    print(f'Dropped Rows: {droppedRows}')

    # Save output and format
    fname = f'{path_save} - [{old_name}] vs [{new_name}].xlsx'
    file = pd.ExcelWriter(fname, engine='openpyxl')
    
    new_copy.to_excel(file, sheet_name='diffs_founded', index=False)
    df_NEW.to_excel(file, sheet_name='new_file', index=False)
    df_OLD.to_excel(file, sheet_name='old_file', index=False)

    # get openpyxl objects
    wb  = file.book
    ws = file.sheets['diffs_founded']
    
    red_fill = PatternFill(start_color='95A7B3', \
                                end_color='95A7B3', fill_type='solid')
    red_font = Font(color='00FF0000', italic=True)
    new_fmt = Font(color='3F976D',bold=True, italic=True)
    removed = Font(color='95A7B3',bold=True, italic=True)

    dxf = DifferentialStyle(font=red_font, fill=red_fill)
    highlight = Rule(type="containsText", operator="containsText", text=">", dxf=dxf)
    highlight.formula = ['SEARCH(">", A1)']
    ws.conditional_formatting.add(f'A1:ZZ{ws.max_row}', highlight)
    
    red_header = PatternFill(start_color='00FF0000', \
                        end_color='00FF0000', fill_type='solid')
    for column in bill_cols:
        dx = DifferentialStyle(font=Font(color='FFFFFF', bold=True), fill=red_header)
        header_style = Rule(type="containsText", operator="containsText", text=column, dxf=dx)
        header_style.formula = [f'SEARCH("{column}", A1)']
        ws.conditional_formatting.add(f'A1:ZZ{ws.max_row}', header_style)
     
    #print(len(newRows))
    for site in new_copy['sites']:
        if site in newRows:
            idx = new_copy.index[new_copy[index_col]==site].to_list()
            idx = list(map(lambda x: x+2, idx))
            change_format(*idx,ws,new_fmt)
        if site in droppedRows:
            idx = new_copy.index[new_copy[index_col]==site].to_list()
            idx = list(map(lambda x: x+2, idx))
            change_format(*idx,ws,removed)
    
    wb.save(fname)
    print('\nDone.\n')

tw_bills = ['Site ID ', 'Strategic','Critical','Site Type (See structure Type)','Core Site Type',\
            'Transmission System','Macro Site - Transmission Hub Site',\
            'Macro Site - Transmission Hub Site with/without Shelters',\
            'Transmission Sites','Room Configuration','Power Supply','Air Conditioning',\
            'Site Status','Strategic_Site_Bucket','CriticalSite_Beyond_10',\
            'Sites_As_Metered_Estimated','Meter_Sharing_Site','Bts_Sites',\
            'Phase1_Site_Or_Consent_Site','Transfer_Date_Of_Consent_Site',\
            'WIP_Sites','Active_Sharing_Arrangement','Subsequent_Sharing_Arrangement',\
            'First_Active_Sharing_Deployment_Type','First_Active_Sharing_Start_Date',\
            'First_Active_Sharing_End_Date']

pathtw='/content/TowerDB_Ireland_20210630 v1.xlsx'
pathold = '/content/TowerDB_Ireland_20210731.csv'
sheet='TowerDB_Input_Ireland_20210630'
skiprows=0
skipcols=0
to_save = '/content/IE_TW'
old_n = 'TowerDB_Ireland_20210630 v1.xlsx'
new_n = "TowerDB_Ireland_20210731.csv"
dates_tw = ['infrastructure ready date']
"""(path_OLD, path_NEW, index_col, bill_cols, status_col, path_save, old_name, new_name, type_file='mix',\
                                kind='tw',dates=[], sheetname='', skipr=0, skipc=0)"""
find_diffs_between_files_tw(pathold, pathtw, 'Site ID ', tw_bills,'Site Status', \
                            to_save, old_n, new_n, type_file='mix',kind='tw', dates=dates_tw, sheetname=sheet)

UIS Comparisons

In [24]:
def find_diffs_between_files_uis(path_OLD, path_NEW, index_col, uis_cols, \
                             path_save, old_name, new_name, type_file='mix', sheetname='', skipr=0, skipc=0):
    def lower_str(columns):
        newlist = list(map(lambda x: x.lower(), columns))
        return newlist

    def fit_df(df, uis_cols):
        fit_cols = lower_str(list(df.columns))
        df.columns = fit_cols
        df = df[uis_cols]
        df = df.dropna(subset=[index_col], axis=0)
        #df = df.drop_duplicates(subset=[index_col], keep='last')
        df[index_col] = df[index_col].astype(str)
        df['sites'] = df[index_col]
        # Aqui ajusta os nan e nat dos DFs, quando não forem arquivos não lidos
        df = df.set_index('sites').fillna('')
        return df
        
    def change_format(index, worksheet, format):
        for cell in worksheet[f"{str(index)}:{str(index)}"]:
            cell.font = format

    uis_cols = lower_str(uis_cols)
    index_col = index_col.lower()
    type_file = type_file.lower()
    if type_file == 'excel':
        df_OLD = pd.read_excel(path_OLD, sheet_name = sheetname, header=0, names=uis_cols, skiprows = skipr).fillna('')
        df_OLD = df_OLD.iloc[:,skipc:]
        df_OLD = fit_df(df_OLD,uis_cols)

        df_NEW = pd.read_excel(path_NEW,sheet_name = sheetname, header=0, names=uis_cols, skiprows = skipr).fillna('')
        df_NEW = df_NEW.iloc[:,skipc:]
        df_NEW = fit_df(df_NEW,uis_cols)

    elif type_file == 'csv':
        df_OLD = pd.read_csv(path_OLD, encoding='latin').fillna('')
        df_OLD = fit_df(df_OLD,uis_cols)

        df_NEW = pd.read_csv(path_NEW, encoding='latin').fillna('')
        df_NEW = fit_df(df_NEW,uis_cols)

    else:
        df_OLD = pd.read_csv(path_OLD, encoding='latin')
        df_OLD = fit_df(df_OLD,uis_cols)

        df_NEW = pd.read_excel(path_NEW,sheet_name = sheetname, skiprows = skipr)
        df_NEW = df_NEW.iloc[:,skipc:]
        df_NEW = fit_df(df_NEW,uis_cols)


    # Perform Diff
    new_copy = df_NEW.copy()
    droppedRows = []
    newRows = []

    cols_OLD = df_OLD.columns
    cols_NEW = df_NEW.columns
    sharedCols = list(set(cols_OLD).intersection(cols_NEW))

    for row in new_copy.index:
        if (row in df_OLD.index) and (row in df_NEW.index):
            for col in sharedCols:
                value_OLD = df_OLD.loc[row,col]
                value_NEW = df_NEW.loc[row,col]
               #Error de a.empty() são sites duplcados e nã poodem estar no index
                if value_OLD == value_NEW:
                    new_copy.loc[row,col] = np.nan
                else:
                    new_copy.loc[row,col] = f'{value_OLD} > {value_NEW}'
        else:
            newRows.append(row)

    new_copy = new_copy.dropna(axis=0, how='all')
    new_copy = new_copy.dropna(axis=1, how='all')

    for row in df_OLD.index:
        if row not in df_NEW.index:
            droppedRows.append(row)
            new_copy = new_copy.append(df_OLD.loc[row,:])
    
    new_copy = new_copy.sort_index(key=lambda x: x.str.lower()).fillna('')
    
    new_copy = new_copy.reset_index()

    print(f'\nNew Rows:     {newRows}')
    print(f'Dropped Rows: {droppedRows}')

    # Save output and format
    fname = f'{path_save} - [{old_name}] vs [{new_name}].xlsx'
    file = pd.ExcelWriter(fname, engine='openpyxl')
    
    new_copy.to_excel(file, sheet_name='diffs_founded', index=False)
    df_NEW.to_excel(file, sheet_name='new_file', index=False)
    df_OLD.to_excel(file, sheet_name='old_file', index=False)

    # get openpyxl objects
    wb  = file.book
    ws = file.sheets['diffs_founded']
    
    red_fill = PatternFill(start_color='95A7B3', \
                                end_color='95A7B3', fill_type='solid')
    red_font = Font(color='00FF0000', italic=True)
    new_fmt = Font(color='3F976D',bold=True, italic=True)
    removed = Font(color='95A7B3',bold=True, italic=True)

    dxf = DifferentialStyle(font=red_font, fill=red_fill)
    highlight = Rule(type="containsText", operator="containsText", text=">", dxf=dxf)
    highlight.formula = ['SEARCH(">", A1)']
    ws.conditional_formatting.add('A1:ZZ10000', highlight)
    
    #print(len(newRows))
    for site in new_copy['sites']:
        if site in newRows:
            idx = new_copy.index[new_copy[index_col]==site].to_list()
            idx = list(map(lambda x: x+2, idx))
            change_format(*idx,ws,new_fmt)
        if site in droppedRows:
            idx = new_copy.index[new_copy[index_col]==site].to_list()
            idx = list(map(lambda x: x+2, idx))
            change_format(*idx,ws,removed)
    
    wb.save(fname)
    print('\nDone.\n')

col_uis = ["Site_ID (Alphanumeric)","BTS site applicable charge (Annual)",\
 "Commercials for sites beyond 10% cap of critical sites",\
 "Total charges to be applied for subsequent active sharing arrangement",\
 "Fixed component of energy bill","Variable component of energy bill",\
 "Energy consumption (for metered site)","Agreed unit price (for metered sites)",\
 "Operator’s active energy charge through estimated model",\
 "Agreed Passive Energy Charge for Outdoor Sites with DC Power",\
 "Energy reduction measures - Final value to be charged",\
 "Legacy site upgrades Or BBU capacity upgrade - Final value to be charged",\
 "BTS site - Site service order withdrawal - Final value to be charged",\
 "Energy infrastructure enhancements - Final value to be charged",\
 "One-off power related Capex costs - Final value to be charged",\
 "Space blocking - Unutilized sites\nIf unutilized then select Yes ",\
 "Decommissioning charges - Equipment removal cost - Final value to be charged",\
 "Site exit fee - Site ID part of delta","Site exit fee - Remaining term of site IDs in delta",\
 "Site power availability - Site IDs where service credits are applicable",\
 "Site power availability - Site IDs for Critical sites Categorization",\
 "Site power availability - Total hours applicable ","Site power availability - Unavailable time",\
 "Site power availability - Count of repeat incidents (if applicable)",\
 "Site cooling - Final value associated with Service Credit applicable",\
 "Site access - Final value associated with Service Credit applicable",\
 "Incident resolution - Final value associated with Service Credit applicable",\
 "BTS sites - Deployment time - Delay","Site modification Completion time - Delay",\
 "Delay In RSCR site modification completion Time","Delay In RSCR BTS site completion time",\
 "Delay in Site Modification Projects","Delay in BTS Projects","Excess of Upgrade Capital Expenditure over Threshold"]
pathuis_n = '/content/UserInput_Ireland_20210731.xlsx'
pathuis_o = '/content/UserInput_Ireland_July 21_06.07.2021.xlsx'
sheetuis='SiteLevel'
skiprows=2
to_save = '/content/IE_UIS'
old_n = 'UIS - Old'
new_n = "UIS - Current"

find_diffs_between_files_uis(pathuis_o, pathuis_n, 'Site_ID (Alphanumeric)', col_uis, \
                        to_save, old_n, new_n, type_file='excel', sheetname=sheetuis, skipr=3)


New Rows:     ['DN370']
Dropped Rows: ['CE006']

Done.



TA Comparison

In [ ]:
def find_diffs_between_files_ta(path_OLD, path_NEW, old_name, new_name, index_col, bill_cols, path_save, type_file='mix', am_cols = [],\
                             cols_int=[], cols_date=[], sheetname='', skipr=0, skipc=0):
    
    def lower_str(columns):
        newlist = list(map(lambda x: x.lower(), columns))
        return newlist
     
    def change_format(index, worksheet, format):
        for cell in worksheet[f"{str(index)}:{str(index)}"]:
            cell.font = format
    
    def fit_df(df, index_col):
        df = df.dropna(subset=[index_col], axis=0)
        df = df.sort_values(by=[index_col, 'tenant/customer '])
        lista=[]
        dico = {}
        for id in df[index_col]:
            nome = str(id)
            if nome not in dico.keys():
                dico[nome] = 1
                n = nome+f'_{dico[nome]}'
                lista.append(n)
            else:
                dico[nome] += 1
                n = nome+f'_{dico[nome]}'
                lista.append(n)
        #print(lista)
        df['sites'] = lista
        df = df.set_index('sites').fillna('')
        return df

    bill_cols = lower_str(bill_cols)
    index_col = index_col.lower()
    type_file = type_file.lower()
    am_cols = lower_str(am_cols)
    cols_int = lower_str(cols_int)
    cols_date = lower_str(cols_date)
    if type_file == 'excel':
        df_OLD = pd.read_excel(path_OLD,sheet_name = sheetname, skiprows = skipr).fillna('')
        df_OLD = df_OLD.iloc[:,skipc:]
        df_OLD = fit_df(df_OLD,bill_cols)

        df_NEW = pd.read_excel(path_NEW,sheet_name = sheetname, skiprows = skipr).fillna('')
        df_NEW = df_NEW.iloc[:,skipc:]
        df_NEW = fit_df(df_NEW,bill_cols)

    elif type_file == 'csv':
        df_OLD = pd.read_csv(path_OLD, encoding='latin').fillna('')
        df_OLD = fit_df(df_OLD,bill_cols)

        #Ajustado só para CZ_Julho, tirei o encoding
        df_NEW = pd.read_csv(path_NEW).fillna('')
        df_NEW = fit_df(df_NEW,bill_cols)

    else:
        ta_col_ord = lower_str(["Site ID ","Site Name ","Address ","Site Provider","Site type",\
              "Tenant/Customer ","Customer Type "," Initial Rent  "," Current Rent  "," REC Uplift Charge ",\
              "Current Power Charge","Renegotiations ","Lease Start Date ","Lease End date ","Right to Renew ",\
              "End of Renewal Period ","Payment Terms ","Payment Frequency ","Rent Review ","Indexation Driver",\
              "Fixed Increase ","CPI ","Percentage","Increase frequency","Increase Date","VAT","Rate ","Termination date "])
        ta_col_ord = lower_str(ta_col_ord)
        df_OLD = pd.read_csv(path_OLD, encoding='ISO-8859-1')
        df_OLD.columns = lower_str(list(df_OLD.columns))
        df_OLD = fit_df(df_OLD, index_col)
        
        """Read New File"""
        df_NEW = pd.read_excel(path_NEW,sheet_name = sheetname, skiprows = skipr)
        df_NEW.columns = lower_str(list(df_NEW.columns))
        df_NEW = df_NEW[ta_col_ord]
        df_NEW = fit_df(df_NEW, index_col)

        for column in cols_date:
            lista=[]
            df_NEW[column] = df_NEW[column].replace(['-','#N/D', 'N/A', np.nan],'')
            for dt in df_NEW[column]:
                if dt != '':
                    new_dt = f"{dt:%d/%m/%Y}"
                    lista.append(new_dt)
                else:
                    lista.append(dt)

            df_NEW[column] = lista

        for i in bill_cols:
            df_NEW[i] = df_NEW[i].replace(['N/A', 'n/a',"0", '_', np.nan,'nan', ' '], '')
        #df_NEW = df_NEW.applymap(lambda x: x.encode('unicode_escape').decode('ISO-8859-1') if isinstance(x, str) else x)

    # Perform Diff
    new_copy = df_NEW.copy()
    droppedRows = []
    newRows = []

    cols_OLD = df_OLD.columns
    cols_NEW = df_NEW.columns
    sharedCols = list(set(cols_OLD).intersection(cols_NEW))

    for row in new_copy.index:
        if (row in df_OLD.index) and (row in df_NEW.index):
            for col in sharedCols:
                value_OLD = str(df_OLD.loc[row,col])
                value_NEW = str(df_NEW.loc[row,col])
               #Error de a.empty() são sites duplcados e nã poodem estar no index
                if value_OLD == value_NEW:
                    new_copy.loc[row,col] = np.nan
                else:
                    new_copy.loc[row,col] = f'{value_OLD} > {value_NEW}'
        else:
            newRows.append(row)

    new_copy = new_copy.dropna(axis=0, how='all')
    new_copy = new_copy.dropna(axis=1, how='all')

    for row in df_OLD.index:
        if row not in df_NEW.index:
            droppedRows.append(row)
            new_copy = new_copy.append(df_OLD.loc[row,:])
    
    new_copy = new_copy.sort_index(key=lambda x: x.str.lower()).fillna('')
    
    new_copy = new_copy.reset_index()


    print(f'\nNew Rows:     {newRows}')
    print(f'Dropped Rows: {droppedRows}')

    new_copy['sites_code'] = [re.sub(r'(\_[0-9]+d{0,2})$', '', site) for site in new_copy['sites']]
    new_copy = new_copy[['sites_code']+ new_copy.columns[:-1].tolist()]

    # Save output and format
    fname = f'{path_save} {old_name} vs {new_name}.xlsx'
    file = pd.ExcelWriter(fname, engine='openpyxl')
    
    new_copy.to_excel(file, sheet_name='diffs_founded', index=False)
    df_NEW.to_excel(file, sheet_name='new_file', index=False)
    df_OLD.to_excel(file, sheet_name='old_file', index=False)

    # get openpyxl objects
    wb  = file.book
    ws = file.sheets['diffs_founded']
    
    red_fill = PatternFill(start_color='95A7B3', \
                                end_color='95A7B3', fill_type='solid')
    red_font = Font(color='00FF0000', italic=True)
    new_fmt = Font(color='3F976D',bold=True, italic=True)
    removed = Font(color='95A7B3',bold=True, italic=True)

    dxf = DifferentialStyle(font=red_font, fill=red_fill)
    highlight = Rule(type="containsText", operator="containsText", text=">", dxf=dxf)
    highlight.formula = ['SEARCH(">", A1)']
    ws.conditional_formatting.add('A1:ZZ10000', highlight)
    
    red_header = PatternFill(start_color='00FF0000', \
                             end_color='00FF0000', fill_type='solid')
    for column in bill_cols:
        dx = DifferentialStyle(font=Font(color='FFFFFF', bold=True), fill=red_header)
        header_style = Rule(type="containsText", operator="containsText", text=column, dxf=dx)
        header_style.formula = [f'SEARCH("{column}", A1)']
        ws.conditional_formatting.add('A1:ZZ10000', header_style)

    #print(len(newRows))
    for site in new_copy['sites']:
        if site in newRows:
            idx = new_copy.index[new_copy['sites']==site].to_list()
            idx = list(map(lambda x: x+2, idx))
            change_format(*idx,ws,new_fmt)
        if site in droppedRows:
            idx = new_copy.index[new_copy['sites']==site].to_list()
            idx = list(map(lambda x: x+2, idx))
            change_format(*idx,ws,removed)
    ws.delete_cols(2)
    wb.save(fname)
    print('\nDone.\n')


ta_col = ["Site ID ","Tenant/Customer ","Customer Type ",\
                "Lease Start Date ","Lease End date "]
path_ta_input = '/content/TowerDB_Ireland_20210630 v1.xlsx'
ta_old = '/content/TA_Input_Ireland_20210731.csv'
ta_save = '/content/IE_TA'
ta_tab = 'TA_Input_Ireland_20210630'
dates_ta = ["Lease Start Date ","Lease End date ", "Increase Date"]
interger_ta = ['Increase frequency']
amount_ta = [" Initial Rent  ", " Current Rent  "]
old_n_ta = 'old_in_month(csv)'
new_n_ta = 'new_file_in_month(xlsx)'

find_diffs_between_files_ta(ta_old, path_ta_input,old_n_ta,new_n_ta,  'Site ID ', ta_col, path_save=ta_save,type_file='mix',am_cols=amount_ta,\
                           cols_int=interger_ta, cols_date=dates_ta, sheetname=ta_tab)


LC Comparison

In [ ]:
def find_diffs_between_files_lc(path_OLD, path_NEW, old_name, new_name, index_col, bill_cols, path_save, type_file='mix',\
                                cols_date=[], sheetname='', skipr=0, skipc=0):
    
    def lower_str(columns):
        newlist = list(map(lambda x: x.lower(), columns))
        return newlist
     
    def change_format(index, worksheet, format):
        for cell in worksheet[f"{str(index)}:{str(index)}"]:
            cell.font = format

    def fit_df(df, index_col):
        df = df.dropna(subset=[index_col], axis=0)
        df = df.sort_values(by=[index_col, 'vendor'])
        lista=[]
        dico = {}
        for id in df[index_col]:
            nome = str(id)
            if nome not in dico.keys():
                dico[nome] = 1
                n = nome+f'_{dico[nome]}'
                lista.append(n)
            else:
                dico[nome] += 1
                n = nome+f'_{dico[nome]}'
                lista.append(n)
        #print(lista)
        df['sites'] = lista
        df = df.set_index('sites').fillna('')
        return df

    bill_cols = lower_str(bill_cols)
    index_col = index_col.lower()
    type_file = type_file.lower()
    cols_date = lower_str(cols_date)
    if type_file == 'excel':
        df_OLD = pd.read_excel(path_OLD,sheet_name = sheetname, skiprows = skipr).fillna('')
        df_OLD = df_OLD.iloc[:,skipc:]
        df_OLD = fit_df(df_OLD,bill_cols)

        df_NEW = pd.read_excel(path_NEW,sheet_name = sheetname, skiprows = skipr).fillna('')
        df_NEW = df_NEW.iloc[:,skipc:]
        df_NEW = fit_df(df_NEW,bill_cols)

    elif type_file == 'csv':
        df_OLD = pd.read_csv(path_OLD, encoding='latin').fillna('')
        df_OLD = fit_df(df_OLD,bill_cols)

        #Ajustado só para CZ_Julho, tirei o encoding
        df_NEW = pd.read_csv(path_NEW).fillna('')
        df_NEW = fit_df(df_NEW,bill_cols)

    else:
        col_order_lc = lower_str(["Site ID ","Site Name ","Address ","Site Provider","Contract Type","Vendor"," Current Rent  ",\
          " Sublet/Uplift "," Total Rent ","Misc Fees ","Sublet Clause ","Conditions ","Uplift for Sublet ",\
          "Lease Start Date ","Lease End date ","Right to Renew ","End of Renewal Period ","Payment Terms ",\
          "Payment Frequency","Rent Review ","Fixed Increase ","Indexation Driver","CPI/Increase Type",\
          "Percentage","Increase Frequency","Increase date","VAT Subject","VAT","Termination date ","Observations"])
        df_OLD = pd.read_csv(path_OLD, encoding='ISO-8859-1')
        df_OLD.columns = lower_str(list(df_OLD.columns))
        df_OLD = fit_df(df_OLD,index_col)
        #print([i for i in list(df_OLD['sites']) if list(df_OLD['sites']).count(i)>1])

        """Read New File"""
        df_NEW = pd.read_excel(path_NEW,sheet_name = sheetname, skiprows = skipr)
        df_NEW.columns = lower_str(list(df_NEW.columns))
        df_NEW = df_NEW[col_order_lc]
        df_NEW = fit_df(df_NEW, index_col)

        df_NEW["uplift for sublet "] = [re.sub('^€', '', str(x)) for x in df_NEW["uplift for sublet "]]
        df_NEW["uplift for sublet "] = df_NEW["uplift for sublet "].replace('-', '')

        #print([i for i in list(df_NEW['sites']) if list(df_NEW['sites']).count(i)>1])
        #['N/A', 'n/a',"0", '-', '_', np.nan,'nan', ' ', 'not available yet'],
        
        df_NEW = df_NEW.fillna('')

        for column in cols_date:
            lista=[]
            for dt in df_NEW[column]:                
                try:
                    if re.match(r"[0-9]{1,2}[\_|\-|\/][0-9]{1,2}[\_|\-|\/][0-9]{1,4}", dt):
                        lista.append(dt)
                    elif dt == '-' or dt =='':
                        lista.append('')
                except:
                    new_dt = f'{dt:%d/%m/%Y}'
                    lista.append(new_dt)


            df_NEW[column] = lista

        #df_NEW = df_NEW.applymap(lambda x: x.encode('unicode_escape').decode('ISO-8859-1') if isinstance(x, str) else x)
        #print(df_NEW.info())
        # Aqui ajusta os nan e nat dos DFs, quando não forem arquivos não lidos

    # Perform Diff
    new_copy = df_NEW.copy()
    droppedRows = []
    newRows = []

    cols_OLD = df_OLD.columns
    cols_NEW = df_NEW.columns
    sharedCols = list(set(cols_OLD).intersection(cols_NEW))

    for row in new_copy.index:

        if (row in df_OLD.index) and (row in df_NEW.index):
            for col in sharedCols:
                value_OLD = str(df_OLD.loc[row,col])
                value_NEW = str(df_NEW.loc[row,col])
               #Error de a.empty() são sites duplcados e nã poodem estar no index
                if value_OLD == value_NEW:
                    new_copy.loc[row,col] = np.nan
                else:
                    new_copy.loc[row,col] = f'{value_OLD} > {value_NEW}'
        else:
            newRows.append(row)

    new_copy = new_copy.dropna(axis=0, how='all')
    new_copy = new_copy.dropna(axis=1, how='all')

    for row in df_OLD.index:
        if row not in df_NEW.index:
            droppedRows.append(row)
            new_copy = new_copy.append(df_OLD.loc[row,:])
    
    new_copy = new_copy.sort_index(key=lambda x: x.str.lower()).fillna('')
    
    new_copy = new_copy.reset_index()

    new_copy['sites_code'] = [re.sub(r'(\_[0-9]+d{0,2})$', '', site) for site in new_copy['sites']]
    new_copy = new_copy[['sites_code']+ new_copy.columns[:-1].tolist()]

    print(f'\nNew Rows:     {newRows}')
    print(f'Dropped Rows: {droppedRows}')

    # Save output and format
    fname = f'{path_save} {old_name} vs {new_name}.xlsx'
    file = pd.ExcelWriter(fname, engine='openpyxl')
    
    new_copy.to_excel(file, sheet_name='diffs_founded', index=False)
    df_NEW.to_excel(file, sheet_name='new_file', index=False)
    df_OLD.to_excel(file, sheet_name='old_file', index=False)

    # get openpyxl objects
    wb  = file.book
    ws = file.sheets['diffs_founded']
    
    red_fill = PatternFill(start_color='95A7B3', \
                                end_color='95A7B3', fill_type='solid')
    red_font = Font(color='00FF0000', italic=True)
    new_fmt = Font(color='3F976D',bold=True, italic=True)
    removed = Font(color='95A7B3',bold=True, italic=True)

    dxf = DifferentialStyle(font=red_font, fill=red_fill)
    highlight = Rule(type="containsText", operator="containsText", text=">", dxf=dxf)
    highlight.formula = ['SEARCH(">", A1)']
    ws.conditional_formatting.add('A1:ZZ10000', highlight)
    
    red_header = PatternFill(start_color='00FF0000', \
                             end_color='00FF0000', fill_type='solid')
    for column in bill_cols:
        dx = DifferentialStyle(font=Font(color='FFFFFF', bold=True), fill=red_header)
        header_style = Rule(type="containsText", operator="containsText", text=column, dxf=dx)
        header_style.formula = [f'SEARCH("{column}", A1)']
        ws.conditional_formatting.add('A1:ZZ10000', header_style)

    #print(len(newRows))
    for site in new_copy['sites']:
        if site in newRows:
            idx = new_copy.index[new_copy['sites']==site].to_list()
            idx = list(map(lambda x: x+2, idx))
            change_format(*idx,ws,new_fmt)
        if site in droppedRows:
            idx = new_copy.index[new_copy['sites']==site].to_list()
            idx = list(map(lambda x: x+2, idx))
            change_format(*idx,ws,removed)

    ws.delete_cols(2)
    wb.save(fname)
    print('\nDone.\n')

lc_ord = ["Site ID ","Site Name ","Address ","Site Provider","Contract Type","Vendor"," Current Rent  ",\
          " Sublet/Uplift "," Total Rent ","Misc Fees ","Sublet Clause ","Conditions ","Uplift for Sublet ",\
          "Lease Start Date ","Lease End date ","Right to Renew ","End of Renewal Period ","Payment Terms ",\
          "Payment Frequency","Rent Review ","Fixed Increase ","Indexation Driver","CPI/Increase Type",\
          "Percentage","Increase Frequency","Increase date","VAT Subject","VAT","Termination date ","Observations"]

lc_col = ['Site ID ',' Total Rent ', 'Lease Start Date ', 'Lease End date ']
path_lc_input = '/content/TowerDB_Ireland_20210630 v1.xlsx'
lc_old = '/content/LC_Input_Ireland_20210731.csv'
lc_save = '/content/IE_LC'
lc_tab = 'LC_Input_Ireland_20210630'
dates_lc = ["Lease Start Date ","Lease End date ", 'increase date', 'end of renewal period ']
old_n = 'old_in_month(csv)'
new_n = 'new_file_in_month(xlsx)'
"""find_diffs_between_files(ta_old, path_ta_input, 'Code', ta_cols, \
                         status_col='', path_save=ta_save, type_file='mix',kind='ta',\
                         kind_col='tenant agreement id - new',k='service type',am_cols = amount_ta,\
                         cols_int=interger, cols_date=dates, sheetname=sheet)"""
find_diffs_between_files_lc(lc_old, path_lc_input,old_n,new_n, 'Site ID ', lc_col, path_save=lc_save,type_file='mix',\
                            cols_date=dates_lc, sheetname=lc_tab)